In [15]:
import pandas as pd

lgb = pd.read_csv('lbg_test_preds_proba.csv')
lgb = lgb.drop(columns=['class_7'])
cat = pd.read_csv('cat_test_preds_proba.csv')

res = pd.read_csv('ch_th_submission.csv')

In [7]:
cat

,class_0,class_1,class_2,class_3,class_4,class_5,class_6
0,0.116398,0.209584,0.248525,0.154204,0.155831,0.067147,0.048312
1,0.154343,0.108905,0.060660,0.059170,0.143563,0.218134,0.255225
2,0.274648,0.410512,0.125278,0.078477,0.050671,0.037560,0.022854
3,0.035119,0.035927,0.044122,0.115335,0.316388,0.330209,0.122900
4,0.127768,0.290462,0.197760,0.168140,0.092762,0.068442,0.054665
...,...,...,...,...,...,...,...
39138,0.519926,0.238174,0.077244,0.043987,0.044227,0.037550,0.038892
39139,0.498756,0.311278,0.090509,0.043698,0.021866,0.018520,0.015374
39140,0.279518,0.476578,0.123298,0.042111,0.031092,0.032858,0.014545
39141,0.516703,0.330005,0.085737,0.031880,0.017722,0.010226,0.007727


In [8]:
lgb

,class_0,class_1,class_2,class_3,class_4,class_5,class_6
0,0.173665,0.215894,0.173304,0.141428,0.155037,0.074378,0.066294
1,0.184034,0.212860,0.087718,0.086971,0.111774,0.086083,0.230560
2,0.259522,0.301450,0.113615,0.112558,0.085620,0.065784,0.061450
3,0.149840,0.171149,0.115629,0.120687,0.182311,0.166779,0.093604
4,0.176486,0.296320,0.143264,0.140674,0.115738,0.064383,0.063135
...,...,...,...,...,...,...,...
39138,0.250195,0.269488,0.111812,0.092402,0.091315,0.076452,0.108336
39139,0.327212,0.250225,0.126488,0.088337,0.086253,0.060658,0.060828
39140,0.271246,0.324664,0.118284,0.086475,0.082826,0.060291,0.056214
39141,0.294484,0.303087,0.108697,0.097466,0.082156,0.059051,0.055058


In [12]:
df_avg = cat * 0.35 + lgb * 0.65
df_avg

,class_0,class_1,class_2,class_3,class_4,class_5,class_6
0,0.153621,0.213685,0.199631,0.145900,0.155315,0.071847,0.060000
1,0.173642,0.176476,0.078248,0.077241,0.122900,0.132301,0.239192
2,0.264816,0.339622,0.117697,0.100630,0.073388,0.055906,0.047941
3,0.109688,0.123821,0.090602,0.118814,0.229238,0.223980,0.103858
4,0.159435,0.294270,0.162337,0.150287,0.107696,0.065804,0.060170
...,...,...,...,...,...,...,...
39138,0.344601,0.258528,0.099713,0.075457,0.074834,0.062836,0.084031
39139,0.387252,0.271594,0.113895,0.072713,0.063717,0.045910,0.044919
39140,0.274141,0.377834,0.120039,0.070948,0.064719,0.050690,0.041630
39141,0.372261,0.312508,0.100661,0.074511,0.059604,0.041962,0.038492


In [17]:
import numpy as np
import pandas as pd

def predict_with_thresholds(df_proba, thresholds, weights_by_class=None):
    """
    Предсказывает классы на основе порогов для DataFrame с вероятностями.
    
    Параметры:
    - df_proba: DataFrame с вероятностями (столбцы: class_0, class_1, ..., class_6).
    - thresholds: Словарь порогов для классов, например {0: 0.24, 1: 0.235, ...}.
    - weights_by_class: Опциональные веса классов для взвешенного выбора (по умолчанию 1.0).
    
    Возвращает:
    - Массив предсказанных классов (int).
    """
    if weights_by_class is None:
        weights_by_class = {i: 1.0 for i in thresholds.keys()}  # нейтральные веса
    
    preds = []
    for _, row in df_proba.iterrows():
        # Находим классы, превысившие порог
        above_threshold = [
            i for i in thresholds.keys() 
            if row[f'class_{i}'] >= thresholds[i]
        ]
        
        if not above_threshold:
            # Если ни один класс не превысил порог, берём argmax
            preds.append(np.argmax(row.values))
        else:
            # Иначе выбираем класс с максимальным (p * weight)
            best_class = max(
                above_threshold,
                key=lambda i: row[f'class_{i}'] * weights_by_class[i]
            )
            preds.append(best_class)
    
    return np.array(preds, dtype=int)

In [21]:
# Задаём пороги (ваши значения)
thresholds = {
    0: 0.24, 
    1: 0.235, 
    2: 0.122, 
    3: 0.05, 
    4: 0.05, 
    5: 0.05, 
    6: 0.05
}

# Опционально: веса классов (если нужны)
weights_by_class = {
    0: 1.0, 
    1: 0.72, 
    2: 0.52, 
    3: 0.37, 
    4: 0.27, 
    5: 0.19, 
    6: 0.14
}

preds_custom = predict_with_thresholds(df_avg, thresholds, weights_by_class)
preds_custom

array([2, 6, 0, ..., 0, 0, 0])

In [23]:
res['avg'] = preds_custom
res[res['target'] != res['avg']]

,client_num,target,avg
5,20,1,2
7,23,2,4
11,30,2,3
24,64,3,4
27,80,1,2
...,...,...,...
39121,109081,2,0
39123,109085,1,0
39130,109106,2,0
39136,109117,2,0


In [ ]:
fin = res
fin['target'] = fin['avg']
fin = fin.drop(columns=['avg'])

,client_num,target
0,0,2
1,10,6
2,11,0
3,14,4
4,16,1
...,...,...
39138,109127,0
39139,109128,0
39140,109130,0
39141,109137,0


In [26]:
fin.to_csv('lgb_cat_35_65.csv', index=False)